<a href="https://colab.research.google.com/github/roncv/MemProtMD/blob/main/filament.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import glob
import shutil
import subprocess
from random import randrange
import os
import sys
from decimal import *

!python3 -m pip install py3dmol
!python3 -m pip install colorama

import py3Dmol
from colorama import Fore
from google.colab import files
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
#@title Define Settings
#@markdown ### Set Membrane Type:
MembraneType = "POPE:POPG:CARDIOLIPIN" #@param ["POPC","POPE:POPG","POPE:POPG:CARDIOLIPIN"]
PE_PG_ratio = 3.5 #@param {type:"slider", min:1, max:10, step:0.1}
CARD_conc = 0.05 #@param {type:"slider", min:0.0, max:0.25, step:0.01}
if CARD_conc == 0.0:
    card = False
#@markdown ---
#@markdown #### Set number of Repeats:
Repeats = 5 #@param {type:"slider", min:1, max:25, step:1}
#@markdown ---
#@markdown #### Rotate Protein:
Rotate = False #@param {type:"boolean"}
#@markdown ---
#@markdown #### Define Distance between Protein and Membrane (nm):
Embed = 2 #@param {type:"slider", min:-3, max:3, step:0.5}
#@markdown ---
#@markdown #### Set Box Dimensions:
Box_Width = 20 #@param {type:"slider", min:8, max:30, step:1}
Box_Height = 20 #@param {type:"slider", min:12, max:50, step:1}
#@markdown #### Add padding to the ends of filaments:
Box_Length_ext = 0 #@param {type:"slider", min:0, max:20, step:1}
#@markdown ---
#@markdown #### Create Filament:
Filament = True #@param {type:"boolean"}
Units = 3 #@param {type:"slider", min:1, max:20, step:1}
Continuous = True #@param {type:"boolean"}

In [ ]:
#@title Upload Protein Structure
def upload_protein():
    """
    Upload a .pdb coordinate file and visualise structure.
    """
    os.chdir('/content/')
    print(Fore.BLUE + "\nUpload Protein Structure:\n")
    upload = files.upload() # request upload of .pdb

    # Arrange working directories
    filename = next(iter(upload))
    name = os.path.splitext(filename)[0]
    working_dir = '/content/' + name +'/'

    if os.path.exists(working_dir):
        shutil.rmtree(working_dir)
    os.makedirs(working_dir)

    os.rename(filename, working_dir + filename)
    os.chdir(working_dir)
    mol1 = open(working_dir + filename, 'r').read()

    mview = py3Dmol.view(width=600,height=400)
    mview.addModel(mol1,'pdb')
    mview.setStyle({'cartoon':{'color':'spectrum'}})
    mview.setBackgroundColor('0xffffff')
    mview.zoomTo()
    mview.show()

    return working_dir, name, filename

working_dir, name, filename = upload_protein() # usage

In [ ]:
# Install Dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/cg2at/"):
  !apt-get update -y
  !apt-get install dssp
  !python3 -m pip install pdb2pqr
  !python3 -m pip install vermouth
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !unzip -o martini_v300.zip
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx


import gromacs
import MDAnalysis
from MDAnalysis import *
from MDAnalysis.lib.distances import self_capped_distance
from MDAnalysis.lib.distances import capped_distance
import numpy as np
from numpy import *
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
#@title Get lipid concentrations
def get_lipid_ratio(pe_pg_ratio=3.5, total=10, card_conc=0.1, card=True, tolerance=0.1):
    """
    Function to get clean a ratio of lipids for a given concentration of cardiolipin.
    """
    card_conc = Decimal(f"{card_conc}")

    # CARD concentration
    card_flt = card_conc * total # float result
    card_rnd = int(round(card_conc * total, 0)) # rounded result
    card_err = np.sqrt(np.square(card_flt-card_rnd)) # difference

    # Fit to tolerance
    r = 0 # counter
    if float(card_err) <= tolerance:
        pass
    else:
        while float(card_err) > tolerance:
            total+=1; r+=1 # counters
            card_flt = card_conc * total
            card_rnd = int(round(card_conc * total, 0))
            card_err = np.sqrt(np.square(card_flt-card_rnd))

    # Set CARD count
    card = card_rnd

    # PE + PG conc
    pe = int(2*(total//(2*(pe_pg_ratio+1)))*pe_pg_ratio)
    pg = total - (pe+card_rnd)

    ratio = [pe, pg, card]

    return ratio

ratio = get_lipid_ratio(pe_pg_ratio=PE_PG_ratio, total=10, card_conc=CARD_conc, card=True, tolerance=0.01) # usage
ratio

In [ ]:
#@title Get area per lipid
def get_area_per_lipid(ratio):
    """
    Function to get the area per lipid for a given lipid ratio.
    """
    # Constants - area per lipid
    pe_apl = 0.63
    pg_apl = 0.67
    cl_apl = 1.23

    # Weighted densities
    pe_wd = ratio[0]/sum(ratio)*pe_apl
    pg_wd = ratio[1]/sum(ratio)*pg_apl
    cl_wd = ratio[2]/sum(ratio)*cl_apl

    # Sum up for avg. density
    avg_apl = sum([pe_wd,pg_wd,cl_wd])

    return avg_apl

#get_area_per_lipid(ratio)

In [ ]:
# ANSI Escape Code styling - for customizing cell output
def esc(codes, string:str):
    """
    Function to generate ANSI escape codes for customizing cell output.
    """
    if isinstance(codes, list): # process list format
        # Generate text with input string, then format as ANSI escape code
        return f"\033[{';'.join(map(str, codes))}m{string}\033[0m"
    if isinstance(codes, int): # process as an integer
        return f'\033[{codes}m{string}\033[0m'
    else:
        raise TypeError("ANSI codes must be provided as a list or integer")

In [ ]:
# Get Cell Dimensions
def get_dimensions(structure):
    """
    Function to get dimensions of a system's cell from a .pdb file.
    """
    # Create universe with structure file...
    u = MDAnalysis.Universe(structure)
    x = round(float(u.dimensions[0]/10)) # in angstroms
    y = round(float(u.dimensions[1]/10))
    z = round(float(u.dimensions[2]/10))
    cube = [x,y,z]

    return x, y, z, cube
    
#x, y, z, cube = get_dimensions('CG-system-extended.gro') # usage

In [ ]:
# Format topology file
def format_topology(topology, Units=1):
    """
    Format a topology file to include all .itp files.
    """
    replacements = {'Protein        1\n':f'ProteinA      {Units}' +'\n','NA+':'NA',
                    'CL-':'CL',
                    '#include "martini_v3.itp"':'#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n'}
    lines = []
    with open(topology) as infile:
        for line in infile:
            for src, target in replacements.items():
                line = line.replace(src, target)
            lines.append(line)
    with open(topology, 'w') as outfile:
        for line in lines:
            outfile.write(line)

#format_topology('topol_test2.top') # usage

In [ ]:
# Position protein
def position_protein_insane(structure, selection='name BB or name SC*',
                            monolayer_width=1.95):
    """
    Finds the COM and minimum z-coordinate of protein structure given. Difference 
    in values is added to the average monolayer width. Made to work with the insane3.py 
    -center and -dm flags, where the functions output used as the -dm value results in 
    the protein just touching the membrane. To embed the protein more or less, increase 
    or decrease the value accordingly.

    Distances in nm.
    """
    # Create universe
    u = MDAnalysis.Universe(structure)

    # Protein dimensions in z-axis
    z_coords_p = u.select_atoms(f'{selection}').positions[:, 2] # select protein
    p_min_z = float(z_coords_p.min())/10.0
    com_p = (u.select_atoms(f'{selection}').center_of_mass()/10.0)[2]
    adjustment = (com_p - p_min_z) + monolayer_width # in nm

    return adjustment

#position_protein_insane('protein-em.pdb') # usage

In [ ]:
# Topology file entries
def insane_topadd(top_add, og_top):
    """
    Function to add new entries to a topology file.
    """
    with open(og_top, "a") as output_file:
        with open(top_add) as input_data:
            # Find additional entries for topology
            for line in input_data:
                if line.strip() == 'Protein        1':
                    break
            for line in input_data:  # read + write them to original file
                output_file.write(line.rstrip('\n').strip()+"\n")

#insane_topadd('topol_ext.top', 'topol1.top') # usage

In [ ]:
#@title Set-up Coarse-Grained Membrane Protein Systems
%%capture
def setup_cg_system(Units=Units, MembraneType=MembraneType, Repeats=Repeats,
                    Rotate=Rotate, ElasticCutoff=0.7, NamingConvention=None, Box_Width=Box_Width,
                    Box_Height=Box_Height, Box_Length_ext=None, Embed=0, Filament=Filament,
                    Continuous=Continuous, pe=7, pg=2, crd=1, Ratio=None):
    """
    Function to set-up coarse-grained membrane-protein systems.
    """
    print(f'\n{esc([100, 97], "INITIALIZE")}\n')

    # Move to working directory
    os.chdir(working_dir)

    # Options
    if NamingConvention == "elastic_cutoff": # ***
        name_nc = f'-eu{ElasticCutoff}'
    elif NamingConvention == None:
        name_nc = ''
    else:
        name_nc = NamingConvention

    # LIPID COMPOSITION
    if Ratio == None: # depreciate this
        Ratio = [pe, pg, crd]
        if crd == 0:
            MembraneType = 'POPE:POPG'
        # Break in the if statement
        if MembraneType == "POPC":
            lipid_name = 'pc'
            lipid = '-l POPC:1'
        elif MembraneType == "POPE:POPG":
            lipid_name = 'pepg'
            lipid = f'-l POPE:{pe} -l POPG:{pg}'
        elif MembraneType == "POPE:POPG:CARDIOLIPIN":
            lipid_name = f'pe{pe}-pg{pg}-c{crd}'
            lipid = f'-l POPE:{pe} -l POPG:{pg} -l CARD:{crd}'
        else:
            print('Invalid lipid ratios!')
    # Using Ratio instead of individual compositions - PREFERRED
    elif isinstance(Ratio, list):
        pe, pg, crd = Ratio
        if crd == 0:
            lipid_name = f'pe{pe}-pg{pg}'
            lipid = f'-l POPE:{pe} -l POPG:{pg}'
        else:
            lipid_name = f'pe{pe}-pg{pg}-c{crd}'
            lipid = f'-l POPE:{pe} -l POPG:{pg} -l CARD:{crd}'
        apl = get_area_per_lipid(Ratio)
        print(f"Lipid Ratio: {Ratio}")
    else:
        raise TypeError("Invalid lipid selection!")

    # Organize Directories - Including naming conventions
    name_filaments = f"f{Units}"
    name_embed = f"eb{Embed}"
    subdir = f"{working_dir}{name}-{lipid_name}-{name_filaments}-{name_embed}{name_nc}/"
    print(subdir)
    shutil.rmtree(subdir, ignore_errors=True)
    os.makedirs(subdir, exist_ok=True)
    shutil.copyfile(filename, subdir+filename)
    os.chdir(subdir)

    # martini .itp files
    for file in glob.glob(r'/content/martini*.itp'):
        #print(file)
        shutil.copy(file, subdir)

    # Adjust cell size based on no. of filaments
    print("\nBuilding cell...\n")
    if Filament == True:
        gromacs.genconf(f=filename,o='ready.pdb',nbox=[Units,1,1],backup=False,renumber=True)
        x, y, z, _ = get_dimensions('ready.pdb')
        if Box_Length_ext is not None or if Box_Length_ext != 0:
            cube = [x+Box_Length_ext, Box_Width, Box_Height] # add x-padding
            Continuous = False # prevent continuous filament with x-padding
        else:
            cube = [x, Box_Width, Box_Height]
    else:
        cube = [Box_Width, Box_Width, Box_Height]
        gromacs.genconf(f=filename,o='ready.pdb',nbox=[1,1,1],backup=False,renumber=True)
    # Print dimensions
    print(f"\nCell dimensions: {cube}")


    # Write selected settings to dir # *** UPDATE AT THE END
    with open('settings.txt', 'w') as file:
            file.write(f'MembraneType = {MembraneType}\nRepeats = {Repeats}\n'
                       f'Rotate = {Rotate}\nEmbed = {Embed}\nBox_Width (y) = {Box_Width}\n'
                       f'Box_Length (x) = {cube[0]}\nBox_Height (z) = {Box_Height}\n
                       f'Box_Length_ext (y) = {Box_Length_ext}\nFilament = {Filament}\nUnits = {Units}\n'
                       f'ElasticBondUpperCutoff = {ElasticCutoff}\nRatio = {Ratio}\nContinuous = {Continuous}')
    # 
    gromacs.make_ndx(f="ready.pdb",o='index.ndx',input=('del 0', 'del 1-100','q'))
    gromacs.editconf(f="ready.pdb",o='protein.pdb',n='index.ndx',
                     c=True,box=cube,input=(0,0),backup=False,label='A',resnr=1)


    # Get dimensions of the box 
    x, y, z, _ = get_dimensions('protein.pdb')
    u = MDAnalysis.Universe('protein.pdb')
    boxed = u.dimensions

    # Format residues
    print("\nFormatting .pdb...")
    with open('protein.pdb', 'r') as file :
        filedata = file.read()
    filedata = filedata.replace('HSE', 'HIS')
    filedata = filedata.replace('HSD', 'HIS')
    filedata = filedata.replace('MSE', 'MET')
    filedata = filedata.replace(' SE ', ' SD ')

    # Update .pdb
    with open('protein.pdb', 'w') as file:
        file.write(filedata)
    # Write EM .mdp
    with open('em.mdp','w') as em:
                em.write('integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001')
    # Write Topology
    with open('topol.top','w') as top:
                top.write('#include "martini_v3.0.0.itp"\n'
                          '#include "martini_v3.0.0_ions_v1.itp"\n'
                          '#include "martini_v3.0.0_solvents_v1.itp"\n'
                          '#include "martini_v3.0.0_phospholipids_v1.itp"\n'
                          f'#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProteinA 1')
    os.system(f'cp topol.top topol_protein.top')

    # GENERATE COARSE GRAINED PROTEIN
    print("\nCoarse graining the protein...")
    merge = 'A' 
    eunit = 'molecule' # 'molecule
    os.system('martinize2 -f protein.pdb -dssp mkdssp -ff martini3001 '
              f'-x protein-cg.pdb -o protein-cg.top -elastic -ef 500 -eu {str(ElasticCutoff)} ' 
              f'-el 0.5 -ea 0 -ep 0 -merge {merge} -maxwarn 100000 -eunit {eunit}') # see eunit + merge (chains...)
    print("Done.")

    os.system("sed -e 's/^molecule.*/ProteinA 1/g' molecule_0.itp | grep -v ';' >  protein-cg.itp")

    with open("protein-cg.itp") as f:
        contents1, constraints, contents2 = f.read().partition("[ constraints ]\n")

    # Restraints - add continuity to filament elastic network
    U = MDAnalysis.Universe('protein-cg.pdb')
    BB = U.select_atoms('name BB')
    d1 = pd.DataFrame(np.column_stack(self_capped_distance(BB.positions,max_cutoff=7,min_cutoff=5,box=boxed,method='pkdtree')))
    d2 = pd.DataFrame(np.column_stack(self_capped_distance(BB.positions,max_cutoff=7,min_cutoff=5,method='pkdtree')))
    zid = pd.DataFrame(BB.ix+1)
    d1 = d1.sort_values(by=[0, 1])
    d2 = d2.sort_values(by=[0, 1])
    df = pd.concat([d1,d2])
    df = df.sort_values(by=[0, 1])
    df = df.drop_duplicates(keep=False,subset=(0,1))
    df[2] = (df[2]/10).map('{:.5f}'.format)
    zi = dict(zip(zid.index,zid[0]))
    df[3] = df[0].replace(zi).map('{:.0f}'.format)
    df[4] = df[1].replace(zi).map('{:.0f}'.format)
    df[5] = 1
    df[6] = 500
    df = df[df[0]<df[1]]
    infinity = df.to_csv(columns=(3, 4, 5, 2, 6),sep='\t', index=False, header=False)

    # Rotation
    if Rotate == True:
        gromacs.editconf(f='protein-cg.pdb',o='protein-cg.pdb',
                         rotate=[randrange(360),randrange(360),0],box=cube)
    else:
        gromacs.editconf(f='protein-cg.pdb',o='protein-cg.pdb', box=cube)

    # CONTINUITY
    # .mdp settings
    if Continuous == True:
        pcoupltype = 'anisotropic'
        compressibility = '3e-4 3e-4 3e-4 0 0 0'
        ref_p = '1.0 1.0 1.0 0 0 0'
        # New .itp parameters
        with open("protein-cg.itp", "w") as new:
            new.write(contents1 + infinity + constraints + contents2)
    else:
        pcoupltype = 'semiisotropic'
        compressibility = '3e-4 3e-4'
        ref_p = '1.0 1.0'

    # ENERGY MINIMIZE PROTEIN
    print("\nPreparing CG protein for energy minimization...")
    gromacs.editconf(f='protein-cg.pdb',o='protein-box.pdb',c=True,box=cube)
    gromacs.grompp(f='em.mdp',o='em.tpr',c='protein-box.pdb',maxwarn='-1',backup=False,v=True)
    gromacs.mdrun(deffnm='em',c='protein-em.pdb',backup=False)

    # Get protein height and calculate membrane insertion depth
    p_height = position_protein_insane('protein-em.pdb')
    Distance = p_height + Embed

    # REPEATS
    for rep in range(1,Repeats+1):
        rep=str(rep)
        print(f"\n{esc(1, f'Repeat {rep}')} ({esc(34, f'CG-system{rep}')})")
        os.chdir(subdir)

        # Add lipids
        print("\nAdding lipids...")
        print(f"Lipid ratio: {Ratio}")
        print(f"Area per lipid: {round(apl, 3)}")
        if Rotate == True:
            os.system(f'python2.7 /content/insane3.py {lipid} -salt 0.15 -sol W -a {apl} '
                      f'-o CG-system{rep}.gro -p topol.top -f protein-em.pdb -center '
                      f'-rotate random -ring -x {x} -y {y} -z {z} -dm {Distance}')
        else:
            os.system(f'python2.7 /content/insane3.py {lipid} -salt 0.15 -sol W -a {apl} '
                      f'-o CG-system{rep}.gro -p topol.top -f protein-em.pdb '
                      f'-center -ring -x {x} -y {y} -z {z} -dm {Distance}')
        print("Done.")

        # Format topology
        format_topology('topol.top')
        os.system(f'cp topol.top topol{rep}.top')


        # ENERGY MINIMIZATION
        print(f"\nPreparing CG system {rep} for energy minimization...")
        gromacs.grompp(f='em.mdp', o=f'em{rep}.tpr', c=f'CG-system{rep}.gro',maxwarn='-1',backup=False,v=True)
        gromacs.mdrun(deffnm=f'em{rep}', c=f'CG-system{rep}.pdb', backup=False)
        gromacs.trjconv(f=f'CG-system{rep}.pdb',o=f'CG-system{rep}.pdb',pbc='mol',
                        s=f'em{rep}.tpr',conect=True,input='0',backup=False)
        print("System minimzed.")

        # EQUILIBRATION
        print(f"\nPreparing CG system {rep} for equillibration...")
        os.makedirs('EQ'+rep, exist_ok=True)
        gromacs.make_ndx(f=f'CG-system{rep}.pdb',o=f'index{rep}.ndx',
                         input=('del 0', 'del 1-40', '0|rPOP*','1&!0','!1','del 1','name 1 Lipid','name 2 SOL_ION','q'),
                         backup=False)
        with open('cgequil.mdp','w') as md: # create equil .mdp
                    md.write('integrator = md\ntinit = 0.0\ndt = 0.02\n'
                             'nsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\n'
                             'nstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\n'
                             'compressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\n'
                             'pbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\n'
                             'rcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\n'
                             'rvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\n'
                             'coulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\n'
                             'epsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\n'
                             'tc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\n'
                             'Pcoupl  = berendsen\nPcoupltype  = semiisotropic\ntau_p  = 12.0\n' 
                             'compressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\n' 
                             'gen_temp  = 310\ngen_seed  = -1\nconstraints  = none\n'
                             'constraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\n'
                             'lincs_warnangle = 30\n')
        gromacs.grompp(f='cgequil.mdp',o=f'EQ{rep}/eq',c=f'CG-system{rep}.pdb',
                       maxwarn=-1, n=f'index{rep}.ndx',backup=False)
        os.chdir(f'{subdir}/EQ{rep}')
        print("Equillibration starting...")
        gromacs.mdrun(deffnm='eq',backup=False, v=True)
        os.chdir(subdir)
        print("Equillibration complete.")

        # SETTING UP A PRODUCTION RUN
        print(f"\nSetting up a production run for CG system {rep}...")
        os.makedirs(f'MD{rep}', exist_ok=True)
        with open('cgmd.mdp','w') as md: # create production .mdp
                    md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 250000000\n'
                             'nstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\n'
                             'nstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\n'
                             'ns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\n'
                             'rcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\n'
                             'vdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\n'
                             'cutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\n'
                             'vdw-modifier  = Potential-shift\nepsilon_rf  = 0\n'
                             'verlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\n'
                             'tc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\n'
                             f'ref_t  = 310 310 310\nPcoupl  = berendsen\nPcoupltype  = {pcoupltype}\n'
                             f'tau_p  = 12.0\ncompressibility = {compressibility}\nref_p  = {ref_p}\n'
                             'gen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\n'
                             'constraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\n'
                             'lincs_warnangle = 30\n')
        
        gromacs.grompp(f='cgmd.mdp',o=f'MD{rep}/md',c=f'EQ{rep}/eq.gro',
                       maxwarn=-1,n=f'index{rep}.ndx',backup=False)
        shutil.rmtree('MD', ignore_errors=True)
        print("Production run setup complete.")
        print(f"\n{esc(34, f'CG-system{rep}')} {esc(92, f'complete.')}")

In [ ]:
# Set up system
setup_cg_system(Units=Units, Repeats=Repeats, Box_Width=Box_Width, Box_Height=Box_Height, Ratio=ratio,
                Continuous=Continuous, Embed=Embed, Box_Length_ext=Box_Length_ext, NamingConvention=None)

#ratio = get_lipid_ratio(pe_pg_ratio=3.5, total=10, card_conc=0.1, card=True, tolerance=0.01)
#setup_cg_system(Units=3, Repeats=1, Box_Width=20, Box_Height=20, Ratio=ratio,
#                Continuous=False, Embed=2, Box_Length_ext=10, NamingConvention=None)

In [ ]:
#@title Visualise System
def visualise_system(_type='gro', waters=False):
    """
    Visualise systems in 3D using Py3Dmol.
    """
    # Access first repeat of each experiment
    if _type == 'pdb':
        systems = glob.glob(os.path.expanduser(f'{working_dir}*/CG-system1.{_type}'))
    else:
        systems = glob.glob(os.path.expanduser(f'{working_dir}*/EQ1/eq.{_type}'))
    # Visualise
    for i in systems:
        print(Fore.RED + '\n'+i+'\n') # convert to f strings ***
        mview = py3Dmol.view(width=600,height=450)
        mol1 = open(i, 'r').read()
        mview.addModel(mol1,f'{_type}')
        mview.setStyle({'cartoon':{'color':'spectrum'}})
        mview.setStyle({'atom':'PO1','atom':'PO2','atom':'PO3','atom':'PO4'},
                       {'sphere':{'color':'0xF5FF1D', 'radius':2.2,'quality':20}}) # lipids
        if waters is True:
            mview.setStyle({'atom':'W'},{'sphere':{'color':'0x5014F5','quality':20}}) # waters
        mview.setStyle({'atom':'BB'},
                       {'sphere':{'color':'0x5FC8F5', 'radius':3,'quality':20}}) # protein
        mview.setBackgroundColor('0xffffff')
        mview.addUnitCell() # only with .pdb files
        mview.zoomTo()
        mview.show()
        
        # Remove temporary files
        for file in glob.glob(r'#*'):
            os.remove(file)
    # Change back to working directory
    os.chdir(working_dir)
    # Remove temporary files
    for file in glob.glob(r'*/#*'):
        os.remove(file)
    
visualise_system('pdb', waters=False) # use .pdb to see unit cell

In [ ]:
#@title Download Zip File
def download(nm=None):
    """
    Download results as a zip file.
    """
    if nm == None:
        os.chdir('/content/')
        os.system(f'zip -r {name}.zip {name}')
        files.download(f'{name}.zip')
    else:
        os.chdir('/content/')
        os.system(f'zip -r {name}-{nm}.zip {name}')
        files.download(f'{name}-{nm}.zip')

download('basic_testing') # usage